In [1]:
import os
from mpi4py import MPI
import coqui

# CoQui computing environemnt: mpi handler and verbosity
mpi = coqui.MpiHandler()
coqui.set_verbosity(mpi, output_level=2, debug_level=0)

--------------------------------------------------------------------------
Ignoring value for oob_tcp_if_exclude on ccqlin065 (10.250.112.0/20: Did not find interface matching this subnet).
(You can safely ignore this message.)
--------------------------------------------------------------------------


# 🔹 Tutorial Summary

In this tutorial, we will learn how to many-body perturbation theories, one of the main features of CoQuí. We will start from an overview on the theoritical formulation, and then followed with explanation on the CoQuí API for MBPT calculations. 

---
In this tutorial, you will learn how to use CoQuí to:
- Perform MBPT calculations
- Understand essential parameters relevant to the accuracy of your simulations.
- Understand the data structure of the output HDF5 file
- Understand how to plot the MBPT band structures

## 1. Many-Body Perturbation Theories (MBPTs)
*Theory and background*

## 1.1 Hartree-Fock approximation
*Theory and background*

## 1.2 GW approximation 
*Theory and background*


# 2. MBPT in CoQuí

A many-body calculation in `CoQui` requires an input `[interaction]`, 
which includes the many-body Hamiltonian expanded in the provided 
single-particle basis.
Additionally, all MBPTs in `CoQui` are finite-temperature calculations 
conducted on the imaginary axis, with the temperature controlled by the 
inverse temperature parameter `beta`. 

All many-body perturbation theories follow the same function API: 
```python
coqui.run_{MBPT_TYPE}(params: dict, h_int: coqui.ThcCoulomb or coqui.CholCoulomb, ...)
```
where `...` denotes optional, solver-specfici parameters. 


### 2.1 Representation on the Imaginary Axis
MBPTs, including Hartree-Fock, in `CoQui` compute dynamic correlation functions 
like the single-particle Green's function and retarded screened interactions on 
the imaginary axis. The efficiency of MBPT calculations thus strongly relies on 
compact representations for these dynamic quantities to capture information across 
the full frequency regime effectively. 

In `CoQui`, both fermionic and bosonic correlation functions are expanded using an 
intermediate representation (IR) basis with sparse sampling on both the imaginary-time 
and Matsubara frequency axes. The IR basis and sampling points are pre-generated using 
the open-source software package [sparse-ir](https://sparse-ir.readthedocs.io/en/latest/index.html). 

The size of the samplings on the imaginary axis is determined solely by the dimensionless 
parameter $\Lambda$ and the user-defined accuracy `iaft_prec`. 
Typically, the value of $\Lambda$ must exceed $\beta\omega_{\mathrm{max}}$ with
$\omega_{\mathrm{max}}$ (a.u.) being the bandwidth of the simulated system.
**It's important for users to ensure that $\Lambda$ is sufficiently large for the simulated 
system at a given temperature.**
After $\Lambda$ is determined, `CoQui` offers three accuracy levels through the parameter `iaft_prec`:
"high" for 1e-15 accuracy, "mid" for 1e-10 accuracy, and "low" for 1e-6 accuracy. 
Roughly speaking, the size of the sampling points scales as $O(\log(\Lambda)\log(1/\epsilon))$ 
where $\epsilon$ corresponds to `iaft_prec`. 

### 🔹 Accuracy & Cost in GW/MBPT: Two Key Compression Layers in CoQuí

The efficiency of CoQuí’s GW (and other MBPT methods) comes from **two compression strategies** with controlled accuracy: 
1. **Dynamic-axis compression via IR basis**:  
   Correlations functions at finite temperature $T$ are represented in the **Intermediate Representation (IR)** basis rather than dense uniform Matsubara grids. The relevant control parameters are set in `gw_params` via
   | Parameter   | Role (intuition)                                                | Effect on IR mesh size                          | Notes                                                                                     |
|-------------|------------------------------------------------------------------|--------------------------------------------------|-------------------------------------------------------------------------------------------|
| `beta`      | Inverse temperature (Ha⁻¹)                                       | Larger `beta` ⇒ **more IR basis functions**      | Set by the physics. Lower $T$ generally results in richer low‑frequency structure  |
| `lambda`    | Dimensionless cutoff ≈ **β × energy window (bandwidth)**         | Larger `lambda` ⇒ **more IR basis functions**    | Choose **≥** the spread of input DFT orbital energies × `beta`.    |
| `iaft_prec` | IR basis precision          | Higher precision ⇒ **more IR basis functions**   | Options: `"low"`, `"medium"`, `"high"`. Higher precision increases cost.    |
      
> **Trade-off:**
> For a given `beta`, Higher `lambda`, and `iaft_prec` → more basis functions → more accurate dynamics at higher computational cost.

2. **Orbital-space compression**:  
   Representing four-index Coulomb integrals and other two-paritcle correlation functions in the **Tensor Hypercontraction (THC)** form. Here, the accuracy is handled in Phase 1 during the construction of the THC Coulomb Hamiltonian (see [here](CoQui-tutorials/02a-THC_Coulomb_Hamiltonian.ipynb) for detailed discussion). 

### 🔹 Facilitate self-consistency convergence
The convergence of the self-consistency loop in MBPTs could sometimes be problematic, particularly for correlated materials. To address that, CoQuí offers several iterative solvers to faciliate the convergence. For some cases, it is often necessary to enable sophisticated iterative solver to guarantee the convergence. 

- Damping

- Direct Inverse Iterative Subspace (DIIS)



### 2.2 GW Variants 

Despite being a perturbative approach, $GW$ is still computationally challenging for realistic quantum materials. Various additional approximation upon the $GW$ equations are generally applied, leading to various variants of $GW$ in different electronic structure package. 

Relying on THC decomposition of the Coulomb Hamiltonian, CoQuí provides a common interface that can smoothly run various variants of $GW$ in a consistent manner, allowing a direct comparison betweeen these methods. 

### 🧪 Exercise 0: Construct ThcCoulomb as inputs of MBPTs

Taking the pregenerated THC Coulomb Hamiltonian, complete the following cell to run a single-shot GW. 

In [2]:
coqui.set_verbosity(mpi, output_level=1, debug_level=0)
# Mf for the target system
mf_params = {
    "prefix": "si", 
    "outdir": "data/qe_inputs/si_222/out", 
    "filetype": "h5",
    "nbnd": 10
}
mf = coqui.make_mf(mpi, mf_params, "qe")

# Construct ThcCoulomb and compute the THC integrals during initialization
thc_params = {
    "init": True,
    "storage": "incore",
    "ecut": 35,
    "thresh": 1e-2
}
thc = coqui.make_thc_coulomb(mf, thc_params)


╔═╗╔═╗╔═╗ ╦ ╦╦  ╔╦╗┬ ┬┌─┐╔═╗┌─┐┬ ┬┬  ┌─┐┌┬┐┌┐ 
║  ║ ║║═╬╗║ ║║   ║ ├─┤│  ║  │ ││ ││  │ ││││├┴┐
╚═╝╚═╝╚═╝╚╚═╝╩   ╩ ┴ ┴└─┘╚═╝└─┘└─┘┴─┘└─┘┴ ┴└─┘

  Algorithm                       = ISDF
  THC integrals access            = incore
  Found precomputed THC integrals = false
  --> CoQuí will compute THC integrals.

  ERI::thc Computation Details
  ----------------------------
  Energy cutoff                = 35.0 a.u. | FFT mesh = (19,19,19), Number of PWs = 2685
  Default Slate block size     = 1024
  Default cholesky block size  = 8
  Threshold                    = 0.01
  Distribution tolerance       = 0.2
  Fraction of memory used for estimation = 0.75
  --> CPU Memory Available: 457873 
 

*********************************************************************
  ISDF - fitting interpolating vectors to pair densities 
*********************************************************************
 - Number of interpolating points: 97
 - Number of q-points in IBZ: 4
 - processor grid: (1,1,1)
 - block 

### 🧪 Exercise 1: Run your first Hartree-Fock calculation in CoQuí

Taking the pregenerated THC Coulomb Hamiltonian, complete the following cell to run a single-shot GW. 

In [7]:
coqui.set_verbosity(mpi, output_level=2, debug_level=0)
hf_params = {
    "restart": False, "output": "hf", 
    "niter": 1,
    "beta": 300, "lambda": 1200, "iaft_prec": "high",
    "iter_alg": {
        "alg": "damping", 
        "mixing": 0.7
    }
}
coqui.run_hf(hf_params, h_int=thc)
# FIXME why do we print e-e interaction kernel that many times? 

  Electron-electron interaction kernel
  ------------------------------------
  type          = coulomb
  ndim          = 3
  cutoff        = 1e-08
  screen_type   = none


╔═╗╔═╗╔═╗ ╦ ╦╦  ┌┬┐┬ ┬┌─┐┌─┐┌┐┌   ┌─┐┌─┐┌─┐
║  ║ ║║═╬╗║ ║║   ││└┬┘└─┐│ ││││───└─┐│  ├┤ 
╚═╝╚═╝╚═╝╚╚═╝╩  ─┴┘ ┴ └─┘└─┘┘└┘   └─┘└─┘└  

  - Maximum iteration number:        1
  - Convergence tolerance:           1e-08
  - SCF output:                      hf.mbpt.h5
  - Restart mode:                    no
  - Total number processors:         1
  - Number of nodes:                 1

  Mesh details on the imaginary axis
  ----------------------------------
  Intermediate Representation
  beta = 300.0 a.u.
  lambda = 10000.0
  prec = 1e-15
  nt_f, nt_b, nw_f, nw_b = 104, 104, 104, 105

  Electron-electron interaction kernel
  ------------------------------------
  type          = coulomb
  ndim          = 3
  cutoff        = 1e-08
  screen_type   = none

* Solving Green's function:
Initial chemical potential (mu) = 0.0, n

In [5]:
coqui.set_verbosity(mpi, output_level=1, debug_level=0)
hf_params = {
    "restart": False, "output": "hf", 
    "niter": 1,
    "beta": 300, "lambda": 1200, "iaft_prec": "high",
    "iter_alg": {
        "alg": "damping", 
        "mixing": 0.7
    }
}
coqui.run_hf(hf_params, h_int=thc)


╔═╗╔═╗╔═╗ ╦ ╦╦  ┌┬┐┬ ┬┌─┐┌─┐┌┐┌   ┌─┐┌─┐┌─┐
║  ║ ║║═╬╗║ ║║   ││└┬┘└─┐│ ││││───└─┐│  ├┤ 
╚═╝╚═╝╚═╝╚╚═╝╩  ─┴┘ ┴ └─┘└─┘┘└┘   └─┘└─┘└  

  - Maximum iteration number:        1
  - Convergence tolerance:           1e-08
  - SCF output:                      hf.mbpt.h5
  - Restart mode:                    no
  - Total number processors:         1
  - Number of nodes:                 1


  DYSON timers
  ------------
    Dyson eqn:                      0.002 sec
      - Sigma(t)->Sigma(w):         0.001 sec
      - Dyson loop:                 0.001 sec
      - Redistribute                0.000 sec
      - Gather:                     0.000 sec


╔═╗╔═╗╔═╗ ╦ ╦╦  ┌┬┐┬ ┬┌─┐ ┬ ┬┌─┐
║  ║ ║║═╬╗║ ║║   │ ├─┤│───├─┤├┤ 
╚═╝╚═╝╚═╝╚╚═╝╩   ┴ ┴ ┴└─┘ ┴ ┴└  

  Hartree, Exchange             = true, true
  Number of spins               = 1
  Number of polarizations       = 1
  Number of bands               = 10
  Number of THC auxiliary basis = 97
  K-points                      = 8 total, 4 in the IBZ
  Diver

### 🧪 Exercise 2: Run your first quasi-particle $G_{0}W_{0}$ calculation in CoQuí

Taking the pregenerated THC Coulomb Hamiltonian, complete the following cell to run a single-shot GW. 
- Check QE output to set a proper `lambda` for the IR basis

### 🧪 Exercise 3: Run your first full frequency $GW$ calculation in CoQuí

Taking the pregenerated THC Coulomb Hamiltonian, complete the following cell to run a single-shot GW. 

### 🧪 Exercise 4: Plot the band structure/spectral functions of $G_{0}W_{0}$ and full frequency $GW$

Taking the pregenerated THC Coulomb Hamiltonian, complete the following cell to run a single-shot GW. 

### 🧪 Exercise 5: Basis set convergence of $GW$

# CoQui HDF5 output structure 

The GW output of CoQui is stored in a single HDF5 archive with the following data structure:

####  Overview
```text
{prefix}.mbpt.h5
├── imaginary_fourier_transform/
├── scf/
└── system/
```

#### `imaginary_fourier_transform/`
```text
imaginary_fourier_transform/
├── beta                  # Inverse temperature β
├── lambda                # Dimensionsless paramter that control the accuracy
├── prec                  # Precision 
├── source                # Source of imaginary-axes grids: `dlr` or `ir` 
├── iwn_mesh/             # Fermionic Matsubara frequency mesh
└── tau_mesh/             # Imaginary-time mesh
```

#### `scf/`
Stores all quantities tracked during the self-consistent iterations of the GW or GW+EDMFT loop. Each iteration is saved as a subgroup iterN, with final_iter indicating the index of the final iteration.
```text
scf/
├── final_iter            # Final iteration index (e.g., 10)
├── iter0/
├── iter1/
│   ├── Dm_skij           # Density matrix (spin, kpoints, bands, bands)
│   ├── G_tskij           # Green's function G(τ, spin, kpoints, bands, bands)
│   ├── F_skij            # Static self-energy (spin, kpoints, bands, bands)
│   ├── Sigma_tskij       # Self-energy Σ(τ, spin, kpoints, bands, bands)
│   └── mu                # Chemical potential
├── iter2/
...
├── iter10/
```